# Topic modeling analysis

#### Description
- All 11 qualitative questions analyzed for key topics
- Number of topics varied within the recommended range of 2 - 5 based on unique key words for each group. This was a subjective process that had to be completed by adjusting the number of components, examing the key words, looking at the intertopic distance in the visualizations and reading the most relevant responses to each topic.
- Non-cleaned text of must prominent topcs was included to make reading easier.

### Import libraries and data

In [378]:
import pandas as pd
import sys
import numpy as np
sys.path.append('../')


import nlp
import wrangle

import nltk

from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore")

In [379]:
pd.set_option('display.max_columns', None)
datadf, dictionarydf = wrangle.wrangle_data(path_prefix='../')

### Viewing the imported data

In [380]:
datadf.head(1)

,job_title,job_taxo,job_id,job_conduct_res,job_analyze_res,job_buy_res_report,job_manage_res_proj,job_observe_res,job_plan_res,job_teach_res,job_advocate_res,job_hire_res_vendor,job_lead_res_team,num_employees,num_researchers,primary_industry,types_res_used,future_res,exp_conduct_res,exp_analyze_res,exp_buy_res_report,exp_manage_res_proj,exp_observe_res,exp_plan_res,exp_teach_res,exp_advocate_res,exp_hire_res_vendor,exp_lead_res_team,research_educ,research_educ_cat,research_educ_desc,learning_talks,learning_read,learning_meetup,learning_workshop,learning_conference,likely_watch_video,likely_internet,likely_book,likely_online_group,likely_colleague,likely_meetup,likely_conference,likely_workshop,factor_speaker,factor_diverse_speak,factor_topics,factor_format_sessions,factor_size,factor_network,factor_variety_attend,factor_code,factor_location,factor_ability_to_pay,how_pick_events,best_event,events_attend_recent,ideal_conference_size,ideal_conference_size_cat,ideal_structure,ideal_structure_cat,ideal_multi_track,ideal_single_track,ideal_unconference,session_keynote,session_long_talk,session_short_talk,session_workshop,session_network,session_social_event,session_qa,sesson_topic_tables,other_conference_types,ideal_topics,ideal_attendees,recommendations,persona_id
resp_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
284,Academic,4,1.0,3,2,0,3,3,3,3,2,1,4,0.0,2.0,maritime,both qualitative and quantitative research,both qualitative and quantitative research,5,5,5,5,5,5,5,5,5,5,"sort of, psychology background.",True,NaN,0,1,1,2,3,1,3,3,2,3,3,4,4,3,3,4,2,3,1,3,1,3,4,NaN,NaN,"CHI, INTERACT",depending on the professional values,7.0,"any, depending on the values",5.0,False,False,False,4,2,2,2,3,2,2,2,NaN,research-practice connections,NaN,research-practice connections,2


In [5]:
datadf.describe()

,job_conduct_res,job_analyze_res,job_buy_res_report,job_manage_res_proj,job_observe_res,job_plan_res,job_teach_res,job_advocate_res,job_hire_res_vendor,job_lead_res_team,exp_conduct_res,exp_analyze_res,exp_buy_res_report,exp_manage_res_proj,exp_observe_res,exp_plan_res,exp_teach_res,exp_advocate_res,exp_hire_res_vendor,exp_lead_res_team,learning_talks,learning_read,learning_meetup,learning_workshop,learning_conference,likely_watch_video,likely_internet,likely_book,likely_online_group,likely_colleague,likely_meetup,likely_conference,likely_workshop,factor_speaker,factor_diverse_speak,factor_topics,factor_format_sessions,factor_size,factor_network,factor_variety_attend,factor_code,factor_location,factor_ability_to_pay,session_keynote,session_long_talk,session_short_talk,session_workshop,session_network,session_social_event,session_qa,sesson_topic_tables
count,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000
mean,2.856749,2.913223,0.402204,2.600551,1.982094,2.931129,1.845730,2.924242,0.776860,1.860882,3.283747,3.323691,0.960055,2.633609,3.084022,3.139118,1.862259,2.906336,1.326446,1.632231,2.074380,2.659780,1.801653,1.451791,1.663912,2.982094,3.500000,3.263085,2.079890,3.053719,2.202479,2.250689,2.360882,2.236915,2.871901,3.652893,2.776860,2.209366,1.998623,2.028926,1.903581,2.944904,3.336088,2.706612,2.754821,2.341598,2.988981,2.173554,1.480716,2.121212,2.431129
std,1.025941,0.939210,0.720686,1.166285,1.134273,0.950177,1.236637,1.056235,0.914143,1.486931,1.571974,1.553165,1.608530,1.702955,1.694565,1.589165,1.684913,1.711475,1.654099,1.674476,0.791318,0.571707,0.836609,0.784165,0.761119,0.960436,0.711967,0.834391,1.213355,0.983242,1.150254,1.143723,1.101378,1.157435,1.071954,0.590183,0.926135,1.168658,1.107341,1.158115,1.263411,1.019666,0.971309,1.124344,0.977229,1.106273,1.109154,1.219458,1.239149,1.179908,1.211472
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,0.000000,2.000000,1.000000,3.000000,1.000000,3.000000,0.000000,0.000000,2.000000,2.000000,0.000000,1.000000,2.000000,2.000000,0.000000,1.000000,0.000000,0.000000,2.000000,2.000000,1.000000,1.000000,1.000000,3.000000,3.000000,3.000000,1.000000,3.000000,1.000000,1.000000,2.000000,1.000000,2.000000,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.000000,1.000000,2.000000
50%,3.000000,3.000000,0.000000,3.000000,2.000000,3.000000,2.000000,3.000000,1.000000,2.000000,4.000000,4.000000,0.000000,2.000000,3.000000,3.000000,1.000000,3.000000,1.000000,1.000000,2.000000,3.000000,2.000000,1.000000,2.000000,3.000000,4.000000,3.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,4.000000,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,4.000000,3.000000,3.000000,3.000000,3.000000,2.000000,1.000000,2.000000,3.000000
75%,4.000000,3.000000,1.000000,3.000000,3.000000,4.000000,3.000000,4.000000,1.000000,3.000000,5.000000,5.000000,1.000000,4.000000,5.000000,5.000000,3.000000,5.000000,2.000000,3.000000,3.000000,3.000000,2.000000,2.000000,2.000000,4.000000,4.000000,4.0000

In [6]:
datadf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 726 entries, 284 to 2
Data columns (total 77 columns):
job_title                    726 non-null category
job_taxo                     726 non-null category
job_id                       723 non-null category
job_conduct_res              726 non-null int64
job_analyze_res              726 non-null int64
job_buy_res_report           726 non-null int64
job_manage_res_proj          726 non-null int64
job_observe_res              726 non-null int64
job_plan_res                 726 non-null int64
job_teach_res                726 non-null int64
job_advocate_res             726 non-null int64
job_hire_res_vendor          726 non-null int64
job_lead_res_team            726 non-null int64
num_employees                723 non-null category
num_researchers              722 non-null category
primary_industry             726 non-null category
types_res_used               690 non-null category
future_res                   609 non-null category
exp_con

# Analysis of qualitative questions

## 5. What is your company or organization's primary industry?

#### Clean and lemmatize the data for this question

In [7]:
datadf.primary_industry = datadf.primary_industry.dropna().apply(nlp.basic_clean)
datadf.primary_industry = datadf.primary_industry.dropna().apply(nlp.lemmatize)

#### Create word count matrix and vector. Settings: 2 word ngrams permitted; words in more than 30% of documents were ignored.

In [150]:
primary_industry_matrix, primary_industry_vector = nlp.create_wordcount_matrix(datadf.primary_industry.dropna(), ngram=(1,3), max_df=.3)
primary_industry_matrix, primary_industry_vector

(<726x180 sparse matrix of type '<class 'numpy.int64'>'
 	with 1239 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

#### Apply LDA method using 4, 6 and 8 components (can be changed) and a random state set to ensure the results can be replicated.

In [329]:
lda5 = LatentDirichletAllocation(n_components= 8, random_state = 42)

In [330]:
lda5.fit(primary_industry_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=8, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [331]:
pyLDAvis.sklearn.prepare(lda5, primary_industry_matrix, primary_industry_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.022258  0.066488       1        1  15.945548
1     -0.156636 -0.217845       2        1  15.516030
7     -0.113215  0.104614       3        1  12.905741
6      0.211751  0.084395       4        1  12.307240
4      0.146570 -0.145802       5        1  12.023908
5     -0.207465 -0.051247       6        1  10.947611
3     -0.081401  0.228426       7        1  10.516563
0      0.178137 -0.069029       8        1   9.837359, topic_info=    Category       Freq                  Term      Total  loglift  logprob
75   Default  49.000000            healthcare  49.000000  30.0000  30.0000
159  Default  41.000000            technology  41.000000  29.0000  29.0000
71   Default  31.000000            government  31.000000  28.0000  28.0000
67   Default  36.000000               fintech  36.000000  27.0000  27.0000
151  Default  38.000000              software  38.000000  26.0000  26.0000
39   Default  31.000000                design  31.000000  25.0000  25.0000
51   Default  36.000000             education  36.000000  24.0000  24.0000
157  Default  36.000000                  tech  36.000000  23.0000  23.0000
142  Default  22.000000                retail  22.000000  22.0000  22.0000
26   Default  33.000000            consulting  33.000000  21.0000  21.0000
65   Default  24.000000             financial  24.000000  20.0000  20.0000
148  Default  37.000000               service  37.000000  19.0000  19.0000
92   Default  16.000000             insurance  16.000000  18.0000  18.0000
86   Default  22.000000              industry  22.000000  17.0000  17.0000
136  Default  22.000000              research  22.000000  16.0000  16.0000
66   Default  18.000000     financial service  18.000000  15.0000  15.0000
47   Default  16.000000             ecommerce  16.000000  14.0000  14.0000
63   Default  14.000000               finance  14.000000  13.0000  13.0000
23   Default  13.000000           consultancy  13.000000  12.0000  12.0000
128  Default  10.000000                public  10.000000  11.0000  11.0000
55   Default  10.000000            enterprise  10.000000  10.0000  10.0000
146  Default   9.000000                sector   9.000000   9.0000   9.0000
118  Default   9.000000             nonprofit   9.000000   8.0000   8.0000
111  Default  14.000000                medium  14.000000   7.0000   7.0000
7    Default   9.000000               banking   9.000000   6.0000   6.0000
107  Default   7.000000             marketing   7.000000   5.0000   5.0000
163  Default   8.000000        transportation   8.000000   4.0000   4.0000
28   Default   8.000000              consumer   8.000000   3.0000   3.0000
143  Default  14.000000                  saas  14.000000   2.0000   2.0000
43   Default  13.000000               digital  13.000000   1.0000   1.0000
..       ...        ...                   ...        ...      ...      ...
82    Topic8   3.134668               hr tech   3.891102   2.1028  -3.6670
80    Topic8   3.134668           hospitality   3.891102   2.1028  -3.6670
95    Topic8   3.134566              learning   3.891086   2.1028  -3.6670
126   Topic8   2.131574  professional service   2.888008   2.0153  -4.0527
125   Topic8   2.131574          professional   2.888008   2.0153  -4.0527
96    Topic8   2.131514    learning education   2.887999   2.0153  -4.0527
152   Topic8   2.131497       software design   2.888000   2.0152  -4.0527
165   Topic8   1.128432            university   2.762394   1.4237  -4.6887
5     Topic8   1.128608                   b2b   2.776405   1.4188  -4.6885
20    Topic8   1.128593         communication   2.778553   1.4180  -4.6885
34    Topic8   1.128154              customer   3.589131   1.1617  -4.6889
74    Topic8   2.131661                health   7.275442   1.0914  -4.0526
151   Topic8   9.384044              software  38.895610   0.8971  -2.5705
143   Topic8   2.131569                  s

#### Topic groups (4)
1. education 
2. fintech 
3. healthcare 
4. Tech 

#### Topic groups (6) 
1. fintech
2. education
3. healthcare
4. technology
5. software
6. consultancy

#### Topic groups (8) - 5 & 8 slight overlap
1. fintech
2. education
3. technology
4. consulting
5. software
6. healthcare
7. government
8. public (sector)


In [12]:
lda_W5 = lda5.transform(primary_industry_matrix)

In [13]:
top_doc_column5 = datadf.primary_industry.dropna()

#### Create second word count matrix. Excludes words in appear in 80%+ of all documents and in 2 or fewer documents. Include bigrams.

In [14]:
word_count_matrix5, count_vect5 = nlp.create_wordcount_matrix(datadf.primary_industry, max_df=0.8, min_df=2, ngram=(1,3))

In [15]:
LDA5a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA5a.fit(word_count_matrix5)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [16]:
lda_H = LDA5a.transform(word_count_matrix5)

In [17]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column5, 3)

Top 3 Documents for Topic 0: 

Document 1
real estate tech

Document 2
real estate tech

Document 3
sotware development across domain such a data science deep learning health tech interaction etc

Top 3 Documents for Topic 1: 

Document 1
design research service and innovation planning aka design planning design ledstrategy

Document 2
market research agency 95 client are in financial service

Document 3
ux research we're a ux research agency a vendor used by fortune 500 company

Top 3 Documents for Topic 2: 

Document 1
commercial real estate

Document 2
commercial real estate

Document 3
architecture engineering construction aec industry manufacturing design medium entertainment industry

Top 3 Documents for Topic 3: 

Document 1
banking retail edu logistic government entertainment tourism aviation

Document 2
financial sector public sector resource technology retail healthcare

Document 3
aerospaceautomativebankingcentral government chemical consumer package goodsdefenceeducation ma

## 6. What types of research do you currently use to make decisions?

In [18]:
datadf.types_res_used = datadf.types_res_used.dropna().apply(nlp.basic_clean)
datadf.types_res_used = datadf.types_res_used.dropna().apply(nlp.lemmatize)

In [231]:
types_res_used_matrix, types_res_used_vector = nlp.create_wordcount_matrix(datadf.types_res_used.dropna(), ngram=(1,3), max_df=.3)
types_res_used_matrix, types_res_used_vector

(<690x1209 sparse matrix of type '<class 'numpy.int64'>'
 	with 7564 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [338]:
lda6 = LatentDirichletAllocation(n_components= 8, random_state = 42)

In [339]:
lda6.fit(types_res_used_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=8, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [340]:
pyLDAvis.sklearn.prepare(lda6, types_res_used_matrix, types_res_used_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.029312  0.047520       1        1  19.583433
6     -0.113839  0.011711       2        1  17.523106
0     -0.090529  0.168515       3        1  16.756191
2      0.247138  0.038751       4        1  11.931896
3      0.091311  0.003273       5        1  11.387523
1     -0.013658 -0.063420       6        1   8.793167
5     -0.031080 -0.177801       7        1   7.067632
7     -0.060030 -0.028549       8        1   6.957051, topic_info=     Category        Freq                         Term       Total  loglift  \
659   Default  143.000000                  qualitative  143.000000  30.0000   
693   Default   74.000000                 quantitative   74.000000  29.0000   
501   Default   35.000000                       market   35.000000  28.0000   
502   Default   28.000000              market research   28.000000  27.0000   
123   Default   44.000000           contextual inquiry   44.000000  26.0000   
369   Default   45.000000                      inquiry   45.000000  25.0000   
960   Default   84.000000                         test   84.000000  24.0000   
442   Default   35.000000          interview usability   35.000000  23.0000   
296   Default   40.000000                   generative   40.000000  22.0000   
1142  Default  142.000000                         user  142.000000  21.0000   
236   Default   38.000000                   evaluative   38.000000  20.0000   
673   Default   26.000000     qualitative quantitative   26.000000  19.0000   
834   Default   36.000000                         sort   36.000000  18.0000   
68    Default   35.000000                    card sort   35.000000  17.0000   
351   Default   16.000000                      indepth   16.000000  16.0000   
297   Default   19.000000        generative evaluative   19.000000  15.0000   
436   Default   33.000000             interview survey   33.000000  14.0000   
352   Default   13.000000            indepth interview   13.000000  13.0000   
656   Default   41.000000                         qual   41.000000  12.0000   
864   Default  113.000000                        study  113.000000  11.0000   
278   Default   49.000000                        focus   49.000000  10.0000   
804   Default   21.000000                    secondary   21.000000   9.0000   
1112  Default  147.000000            usability testing  147.000000   8.0000   
445   Default   19.000000  interview usability testing   19.000000   7.0000   
279   Default   44.000000                  focus group   44.000000   6.0000   
1155  Default   34.000000                user research   34.000000   5.0000   
67    Default   61.000000                         card   61.000000   4.0000   
1102  Default   37.000000               usability test   37.000000   3.0000   
1164  Default   26.000000                 user testing   26.000000   2.0000   
1173  Default   26.000000                           ux   26.000000   1.0000   
...       ...         ...                          ...         ...      ...   
249    Topic8    3.206971                       expert    5.880576   2.0591   
1179   Topic8    7.758788                   validation   14.238919   2.0583   
592    Topic8    1.746779      participant observation    3.310891   2.0260   
495    Topic8    5.035158                          map   10.164873   1.9629   
594    Topic8    5.422435                participatory   13.541456   1.7502   
919    Topic8    4.918795           survey focus group   12.093629   1.7658   
918    Topic8    4.918795                 survey focus   12.093629   1.7658   
262    Topic8    5.024429                     feedback   13.574226   1.6716   
591    Topic8    3.390749                  participant    7.413809   1.8831   
456    Topic8    2.568730                  journey map    5.120616   1.9756   
618    Topic8    2.739036                         poll    5.885492   1.9005   
595    Topic8    3.6

#### Topic groups (4)
1. card sort
2. contextual inquiry
3. focus group 
4. market rsearch

#### Topic groups (6) 
1. diary study
2. card (sort)
3. contextual inquiry
4. quantitative survey
5. focus (group)
6. generative (evaluative)

#### Topic groups (8) - 8 overlaps with 2 & 6
1. diary (study)
2. contextual inquiry
3. interview usability
4. gernative evaluative
5. market (research)
6. ux research
7. indepth (interview)
8. concept validation

In [23]:
lda_W6 = lda6.transform(types_res_used_matrix)

In [24]:
top_doc_column6 = datadf.types_res_used.dropna()

In [25]:
word_count_matrix6, count_vect6 = nlp.create_wordcount_matrix(datadf.types_res_used.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [26]:
LDA6a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA6a.fit(word_count_matrix6)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [27]:
lda_H = LDA6a.transform(word_count_matrix6)

In [28]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column6, 3)

Top 3 Documents for Topic 0: 

Document 1
qualitative interview survey card sort prototype testing usability testing analytics etc

Document 2
usability testing moderated and unmoderated card sort tree jacking max diff jtbd odi survey observational ethnographic research concept validation others a needed

Document 3
usability testing empathy1x1 interview survey card sort tree test prototype and concept testing diary study eye tracking

Top 3 Documents for Topic 1: 

Document 1
user research participant observation interview contextual enquiry usability testing feedback analysis data analysis survey

Document 2
contextual inquiry interview survey feedback form quant behavior analysis quant usability study usability testing ux workshop

Document 3
foundational product research usability testing 11 interview unmoderated user testing using usertestingcom survey contextual inquiry diary study analytics

Top 3 Documents for Topic 2: 

Document 1
qual research interview in lab moderated usabi

## 7. What types of research are you considering in the future?

In [29]:
datadf.future_res = datadf.future_res.dropna().apply(nlp.basic_clean)
datadf.future_res = datadf.future_res.dropna().apply(nlp.lemmatize)

In [227]:
future_res_matrix, future_res_vector = nlp.create_wordcount_matrix(datadf.future_res.dropna(), ngram=(1,3), max_df=.3)
future_res_matrix, future_res_vector

(<609x540 sparse matrix of type '<class 'numpy.int64'>'
 	with 3008 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [347]:
lda7 = LatentDirichletAllocation(n_components=8, random_state = 42)

In [348]:
lda7.fit(future_res_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=8, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [349]:
pyLDAvis.sklearn.prepare(lda7, future_res_matrix, future_res_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.201092  0.088563       1        1  17.552875
0     -0.051725  0.138593       2        1  16.424723
1     -0.241193 -0.022545       3        1  15.444785
5      0.133479 -0.075148       4        1  15.399785
6     -0.037241  0.126340       5        1  11.840248
3     -0.012154 -0.154187       6        1   9.976906
2     -0.020680  0.006690       7        1   7.532690
7      0.028422 -0.108306       8        1   5.827987, topic_info=    Category        Freq                Term       Total  loglift  logprob
371  Default  139.000000            research  139.000000  30.0000  30.0000
90   Default   40.000000               diary   40.000000  29.0000  29.0000
91   Default   36.000000         diary study   36.000000  28.0000  28.0000
434  Default   70.000000               study   70.000000  27.0000  27.0000
459  Default   91.000000             testing   91.000000  26.0000  26.0000
12   Default   30.000000           analytics   30.000000  25.0000  25.0000
76   Default   43.000000                data   43.000000  24.0000  24.0000
360  Default   29.000000              remote   29.000000  23.0000  23.0000
343  Default   36.000000        quantitative   36.000000  22.0000  22.0000
501  Default   53.000000           usability   53.000000  21.0000  21.0000
106  Default   20.000000        ethnographic   20.000000  20.0000  20.0000
10   Default   22.000000            analysis   22.000000  19.0000  19.0000
506  Default   32.000000   usability testing   32.000000  18.0000  18.0000
0    Default   20.000000                  ab   20.000000  17.0000  17.0000
162  Default   11.000000               group   11.000000  16.0000  16.0000
137  Default   13.000000               focus   13.000000  15.0000  15.0000
514  Default   38.000000                user   38.000000  14.0000  14.0000
330  Default   26.000000         qualitative   26.000000  13.0000  13.0000
2    Default   18.000000          ab testing   18.000000  12.0000  12.0000
255  Default   46.000000              method   46.000000  11.0000  11.0000
275  Default   18.000000                 new   18.000000  10.0000  10.0000
133  Default   13.000000               field   13.000000   9.0000   9.0000
481  Default   11.000000            tracking   11.000000   8.0000   8.0000
228  Default   19.000000                like   19.000000   7.0000   7.0000
111  Default   17.000000         ethnography   17.000000   6.0000   6.0000
138  Default   10.000000         focus group   10.000000   5.0000   5.0000
494  Default   29.000000         unmoderated   29.000000   4.0000   4.0000
491  Default   13.000000                type   13.000000   3.0000   3.0000
204  Default   29.000000           interview   29.000000   2.0000   2.0000
443  Default    8.000000                sure    8.000000   1.0000   1.0000
..       ...         ...                 ...         ...      ...      ...
220   Topic8    3.294492                 lab    7.418981   2.0307  -4.0108
147   Topic8    2.496026              future    5.811245   1.9974  -4.2884
12    Topic8   11.876368           analytics   30.795286   1.8897  -2.7285
10    Topic8    8.434114            analysis   22.464135   1.8629  -3.0708
111   Topic8    6.462503         ethnography   17.553609   1.8433  -3.3371
113   Topic8    0.898591          evaluative    2.471978   1.8306  -5.3100
219   Topic8    0.898501                know    2.471981   1.8305  -5.3101
416   Topic8    0.898622                soon    2.493755   1.8218  -5.3100
486   Topic8    0.898560               trend    2.493759   1.8217  -5.3101
294   Topic8    0.898557             outcome    2.493759   1.8217  -5.3101
11    Topic8    0.898527      analysis usage    2.493762   1.8217  -5.3101
141   Topic8    0.898520                form    2.493762   1.8217  -5.3101
101   Topic8    0.898512                 don    2.493762   1.8217  -5.3101
527   Topic8    0.898508          validati

#### Topic groups (4)
1. ab testing
2. quantitative
3. diary study
4. participatory

#### Topic groups (6) - 4 & 6 overlap
1. quantitative
2. usability testing
3. field
4. analytics
5. diary study
6. journey

#### Topic groups (8) - 5 is almost completely within 2; 6 & 8 overlap
1. quantitative
2. unmoderated usability
3. diary study
4. ab testing
5. ux
6. focus group
7. ethnographic
8. field study


In [34]:
lda_W7 = lda7.transform(future_res_matrix)

In [35]:
top_doc_column7 = datadf.future_res.dropna()

In [36]:
word_count_matrix7, count_vect7 = nlp.create_wordcount_matrix(datadf.future_res.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [37]:
LDA7a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA7a.fit(word_count_matrix7)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [38]:
lda_H = LDA7a.transform(word_count_matrix7)

In [39]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column7, 3)


Top 3 Documents for Topic 0: 

Document 1
user experience research workshop survey interview usability testing benchmark card sorting etc and i would like to delve deeper in analytics

Document 2
doing more strategic piece to help inform the product strategy early on before the okrs are set have fundamental understanding our user in different market since we have a global product and we havent done a lot of thinking around localized content and market difference doing information architecture research

Document 3
abn multivariate testing analytic research behavioural research card sortingtree testing content auditing experience mapping information architecture ia lab testing build persona development research participant recruitment management scenariotask mapping survey design analysis usability testing userstakeholder interview workshop

Top 3 Documents for Topic 1: 

Document 1
much more usability testing heatmappingbehavior tracking along the line of what pendoio offer exploratory 

## 10. Describe your educational background with research

In [40]:
datadf.research_educ_desc = datadf.research_educ_desc.dropna().apply(nlp.basic_clean)
datadf.research_educ_desc = datadf.research_educ_desc.dropna().apply(nlp.lemmatize)

In [223]:
research_educ_desc_matrix, research_educ_desc_vector = nlp.create_wordcount_matrix(datadf.research_educ_desc.dropna(), ngram=(1,3), max_df=.3)
research_educ_desc_matrix, research_educ_desc_vector

(<540x1118 sparse matrix of type '<class 'numpy.int64'>'
 	with 6604 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [353]:
lda10 = LatentDirichletAllocation(n_components= 4, random_state = 42)

In [354]:
lda10.fit(research_educ_desc_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=4, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [355]:
pyLDAvis.sklearn.prepare(lda10, research_educ_desc_matrix, research_educ_desc_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.021395 -0.019895       1        1  32.467903
3     -0.060932 -0.122454       2        1  25.847322
0      0.168490  0.017193       3        1  20.886478
2     -0.086164  0.125155       4        1  20.798297, topic_info=     Category        Freq                       Term       Total  loglift  \
590   Default  129.000000                     master  129.000000  30.0000   
235   Default  101.000000                     degree  101.000000  29.0000   
752   Default   70.000000                 psychology   70.000000  28.0000   
205   Default   90.000000                     course   90.000000  27.0000   
692   Default   74.000000                        phd   74.000000  26.0000   
594   Default   39.000000              master degree   39.000000  25.0000   
1051  Default   65.000000                         ux   65.000000  24.0000   
349   Default   18.000000               experimental   18.000000  23.0000   
870   Default   45.000000                    science   45.000000  22.0000   
144   Default   21.000000                  cognitive   21.000000  21.0000   
896   Default   26.000000                     social   26.000000  20.0000   
402   Default   29.000000                       grad   29.000000  19.0000   
903   Default   22.000000                  sociology   22.000000  18.0000   
302   Default   32.000000                        did   32.000000  17.0000   
561   Default   16.000000                         ma   16.000000  16.0000   
1105  Default   22.000000                   workshop   22.000000  15.0000   
900   Default   15.000000             social science   15.000000  14.0000   
493   Default   29.000000                interaction   29.000000  13.0000   
258   Default  160.000000                     design  160.000000  12.0000   
351   Default   12.000000    experimental psychology   12.000000  11.0000   
167   Default   12.000000                   computer   12.000000  10.0000   
403   Default   24.000000                grad school   24.000000   9.0000   
36    Default   33.000000               anthropology   33.000000   8.0000   
944   Default   75.000000                      study   75.000000   7.0000   
322   Default   21.000000                  education   21.000000   6.0000   
159   Default   11.000000                  completed   11.000000   5.0000   
373   Default   19.000000                      focus   19.000000   4.0000   
453   Default   10.000000              humancomputer   10.000000   3.0000   
454   Default   10.000000  humancomputer interaction   10.000000   2.0000   
422   Default   40.000000                        hci   40.000000   1.0000   
...       ...         ...                        ...         ...      ...   
523    Topic4   12.045714                        lab   17.076353   1.2213   
752    Topic4   36.062525                 psychology   70.129814   0.9052   
1040   Topic4    5.492489          usability testing    7.696172   1.2330   
692    Topic4   35.614723                        phd   74.853810   0.8275   
887    Topic4    4.364024                    seminar    5.960067   1.2586   
391    Topic4    4.877986                    general    6.839779   1.2323   
1051   Topic4   26.472746                         ux   65.521240   0.6640   
511    Topic4   13.870524                        job   31.994367   0.7345   
983    Topic4    6.360550                    testing   11.196895   1.0048   
630    Topic4   25.084920                     method   80.876735   0.3996   
444    Topic4   12.808509               human factor   30.922204   0.6889   
359    Topic4   12.732868                     factor   31.812264   0.6546   
187    Topic4   11.191016                 conference   26.605602   0.7043   
89     Topic4   10.498027                       book   24.095586   0.7395   
439    Topic4   15.021575                      human   47.871403   0.4113   
116    Topic4 

#### Topic groups (4) - no overlapping circles
1. participated
2. grad school
3. master degree
4. pyschology

#### Topic groups (6) - 1 &4, 2 & 3 overlap
1. participated
2. design research
3. social science
4. grad school
5. human factor
6. master degree

#### Topic groups (8) - extreme overlap between topics 1, 2, 4 & 8, 54% of all the responses

In [45]:
lda_W10 = lda10.transform(research_educ_desc_matrix)

In [46]:
top_doc_column10 = datadf.research_educ_desc.dropna()

In [47]:
word_count_matrix10, count_vect10 = nlp.create_wordcount_matrix(datadf.research_educ_desc.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [48]:
LDA10a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA10a.fit(word_count_matrix10)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [49]:
lda_H = LDA10a.transform(word_count_matrix10)

In [50]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column10, 3)

Top 3 Documents for Topic 0: 

Document 1
i took professional development course in research up to one year in length at undergraduate level and then started postgraduate level private tuition i'd love to do a master or phd degree but am not able to manage the commitment even parttime alongside working full time i continue to take short course to increase my knowledge and develop my skill

Document 2
phd in human computer interaction research method data analysis coursework in various area eg ethnography statistic a well a supporting other people's research and eventually running my own research with supervision and advice from more senior people

Document 3
undergraduate degree in psychology extensive study of behavioral and scientific research methodology and analysis qual and quant interned and worked in lab for several year genetics neuroethology biopsychology graduate degree in architecture extensive study and practice of many type of research method

Top 3 Documents for Topic 1: 

## 14. How do you decide which events to attend?

In [51]:
datadf.how_pick_events = datadf.how_pick_events.dropna().apply(nlp.basic_clean)
datadf.how_pick_events = datadf.how_pick_events.dropna().apply(nlp.lemmatize)

In [265]:
how_pick_events_matrix, how_pick_events_vector = nlp.create_wordcount_matrix(datadf.how_pick_events.dropna(), ngram=(1,3), max_df=.3)
how_pick_events_matrix, how_pick_events_vector

(<644x1332 sparse matrix of type '<class 'numpy.int64'>'
 	with 7899 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [356]:
lda14 = LatentDirichletAllocation(n_components= 8, random_state = 42)

In [357]:
lda14.fit(how_pick_events_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=8, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [358]:
pyLDAvis.sklearn.prepare(lda14, how_pick_events_matrix, how_pick_events_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.059080 -0.014795       1        1  17.476238
2      0.066471  0.043234       2        1  16.726491
5      0.001964  0.013352       3        1  12.492820
1      0.013295  0.084182       4        1  11.609439
6     -0.134272 -0.064325       5        1  11.506188
3      0.025425  0.040555       6        1  10.721741
4      0.065494 -0.139758       7        1  10.459890
0     -0.097457  0.037555       8        1   9.007192, topic_info=     Category        Freq            Term       Total  loglift  logprob
410   Default  193.000000           event  193.000000  30.0000  30.0000
206   Default  194.000000      conference  194.000000  29.0000  29.0000
133   Default   57.000000          budget   57.000000  28.0000  28.0000
262   Default  144.000000            cost  144.000000  27.0000  27.0000
911   Default   40.000000           price   40.000000  26.0000  26.0000
328   Default   17.000000         depends   17.000000  25.0000  25.0000
1155  Default  166.000000           topic  166.000000  24.0000  24.0000
71    Default  101.000000          attend  101.000000  23.0000  23.0000
1064  Default   93.000000         speaker   93.000000  22.0000  22.0000
717   Default   38.000000            look   38.000000  21.0000  21.0000
1136  Default   86.000000            time   86.000000  20.0000  20.0000
695   Default  106.000000        location  106.000000  19.0000  19.0000
191   Default   47.000000         company   47.000000  18.0000  18.0000
684   Default   42.000000           local   42.000000  17.0000  17.0000
700   Default   10.000000   location cost   10.000000  16.0000  16.0000
96    Default   16.000000            away   16.000000  15.0000  15.0000
1301  Default   79.000000            work   79.000000  14.0000  14.0000
389   Default   39.000000        employer   39.000000  13.0000  13.0000
1179  Default   11.000000   topic speaker   11.000000  12.0000  12.0000
569   Default   17.000000              im   17.000000  11.0000  11.0000
984   Default   18.000000      reputation   18.000000  10.0000  10.0000
976   Default   33.000000        relevant   33.000000   9.0000   9.0000
1171  Default    9.000000  topic location    9.000000   8.0000   8.0000
363   Default   25.000000             don   25.000000   7.0000   7.0000
819   Default   28.000000     opportunity   28.000000   6.0000   6.0000
973   Default   13.000000       relevance   13.000000   5.0000   5.0000
308   Default   23.000000             day   23.000000   4.0000   4.0000
749   Default   13.000000          matter   13.000000   3.0000   3.0000
1151  Default   25.000000          timing   25.000000   2.0000   2.0000
600   Default   32.000000     interesting   32.000000   1.0000   1.0000
...       ...         ...             ...         ...      ...      ...
1136   Topic8   16.954946            time   86.229017   0.7807  -3.8046
262    Topic8   23.769506            cost  144.557560   0.6019  -3.4667
448    Topic8    5.020055         expense   15.575789   1.2749  -5.0217
509    Topic8    3.304404         funding    8.459395   1.4671  -5.4399
29     Topic8    2.576499        advanced    5.922720   1.5748  -5.6887
1301   Topic8   13.229315            work   79.872510   0.6092  -4.0527
191    Topic8    9.172904         company   47.326276   0.7663  -4.4189
1155   Topic8   16.717696           topic  166.045925   0.1114  -3.8187
250    Topic8    7.458520         content   37.820629   0.7836  -4.6258
1190   Topic8   14.193775          travel  135.847474   0.1484  -3.9823
649    Topic8    5.049769        learning   18.850514   1.0899  -5.0158
695    Topic8   12.405031        location  106.972152   0.2527  -4.1170
976    Topic8    5.874564        relevant   33.166657   0.6762  -4.8645
1151   Topic8    5.022435          timing   25.631113   0.7773  -5.0212
858    Topic8    3.401052          paying   11.152465   1.2196  -5.4110
491    Topic8    2.576182        

#### Topic groups (4)
1. pay
2. topic
3. location
4. cost

#### Topic groups (6) - overlap between 3 & 6
1. pay
2. value
3. price
4. reputation
5. networking
6. relevance

#### Topic groups (8) - small amount of overlap among 2, 3, 4, & 6
1. design
2. affordable
3. speaker topic
4. reputation
5. location cost
6. value
7. location price
8. time away

In [56]:
lda_W14 = lda14.transform(how_pick_events_matrix)

In [57]:
top_doc_column14 = datadf.how_pick_events.dropna()

In [58]:
word_count_matrix14, count_vect14 = nlp.create_wordcount_matrix(datadf.how_pick_events.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [59]:
LDA14a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA14a.fit(word_count_matrix14)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [60]:
lda_H = LDA14a.transform(word_count_matrix14)

In [61]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column14, 3)

Top 3 Documents for Topic 0: 

Document 1
it is usually a combination of date and my availability to travel the location the size the speaker list and whether the event seems likely to cater to more senior researcher or designer i'm at a point where traveling far for 101 level talk is just not useful to me

Document 2
depends on the cost of the event eg is it an evening meetup v a conference if it's the latter then it would depend on company training budget or depending on how relevant the topic are for my personal development goal also depending on the talk itself eg what the topic are which company are presenting i am more interested in advanced topic around strategy prioritisation or maybe new method

Document 3
location is the primary factor to consider because it will indicate whether i can afford to attend or not my company will cover cost of entry to a conference or workshop but will not cover travel or accommodation i also use personal vacation time to attend learning session s

## 15. What was the best professional learning experience you've ever had?  What made it great?

In [62]:
datadf.best_event = datadf.best_event.dropna().apply(nlp.basic_clean)
datadf.best_event = datadf.best_event.dropna().apply(nlp.lemmatize)

In [282]:
best_event_matrix, best_event_vector = nlp.create_wordcount_matrix(datadf.best_event.dropna(), ngram=(1,3), max_df=.3)
best_event_matrix, best_event_vector

(<594x1630 sparse matrix of type '<class 'numpy.int64'>'
 	with 8977 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [362]:
lda15 = LatentDirichletAllocation(n_components= 4, random_state = 42)

In [363]:
lda15.fit(best_event_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=4, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [364]:
pyLDAvis.sklearn.prepare(lda15, best_event_matrix, best_event_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.011475  0.103204       1        1  30.136914
0      0.049404 -0.042172       2        1  29.573841
3      0.047789 -0.036643       3        1  23.504913
1     -0.108668 -0.024389       4        1  16.784331, topic_info=     Category        Freq            Term       Total  loglift  logprob
1465  Default   22.000000         variety   22.000000  30.0000  30.0000
309   Default    8.000000         covered    8.000000  29.0000  29.0000
329   Default   58.000000             day   58.000000  28.0000  28.0000
1231  Default   13.000000     small group   13.000000  27.0000  27.0000
1210  Default   31.000000         session   31.000000  26.0000  26.0000
1394  Default    8.000000   topic covered    8.000000  25.0000  25.0000
1152  Default   86.000000        research   86.000000  24.0000  24.0000
1079  Default   10.000000         program   10.000000  23.0000  23.0000
1393  Default   58.000000           topic   58.000000  22.0000  22.0000
924   Default   27.000000      networking   27.000000  21.0000  21.0000
120   Default    9.000000       attending    9.000000  20.0000  20.0000
1438  Default   95.000000              ux   95.000000  19.0000  19.0000
1205  Default   19.000000         service   19.000000  18.0000  18.0000
1431  Default   16.000000            user   16.000000  17.0000  17.0000
1113  Default   23.000000            real   23.000000  16.0000  16.0000
559   Default   47.000000            good   47.000000  15.0000  15.0000
303   Default   36.000000          course   36.000000  14.0000  14.0000
1468  Default    6.000000   variety topic    6.000000  13.0000  13.0000
1591  Default  134.000000        workshop  134.000000  12.0000  12.0000
1293  Default   13.000000          strive   13.000000  11.0000  11.0000
248   Default  183.000000      conference  183.000000  10.0000  10.0000
350   Default   76.000000          design   76.000000   9.0000   9.0000
1618  Default   17.000000        year ago   17.000000   8.0000   8.0000
1324  Default    6.000000            sxsw    6.000000   7.0000   7.0000
1206  Default   15.000000  service design   15.000000   6.0000   6.0000
859   Default    9.000000          master    9.000000   5.0000   5.0000
1614  Default   15.000000           world   15.000000   4.0000   4.0000
1167  Default   23.000000      researcher   23.000000   3.0000   3.0000
817   Default   12.000000           local   12.000000   2.0000   2.0000
1626  Default   10.000000           young   10.000000   1.0000   1.0000
...       ...         ...             ...         ...      ...      ...
526    Topic4    2.644637            food    4.140965   1.3363  -6.4055
559    Topic4   14.935997            good   47.440290   0.6290  -4.6742
61     Topic4    2.650677       afternoon    4.157643   1.3346  -6.4032
1147   Topic4    4.293033        remember    8.311022   1.1241  -5.9210
812    Topic4    4.308090       listening    8.369711   1.1206  -5.9175
1167   Topic4    8.101961      researcher   23.659729   0.7131  -5.2859
303    Topic4   10.474299          course   36.493503   0.5365  -5.0291
350    Topic4   16.390489          design   76.538189   0.2436  -4.5813
577    Topic4   20.451722           great  116.241055   0.0471  -4.3599
329    Topic4   12.476440             day   58.445922   0.2405  -4.8542
376    Topic4    7.326120       different   22.045209   0.6831  -5.3866
1614   Topic4    5.846003           world   15.147547   0.8326  -5.6122
1206   Topic4    5.910922  service design   15.957627   0.7916  -5.6012
966    Topic4    4.836546          online   10.984413   0.9644  -5.8018
1591   Topic4   16.287980        workshop  134.055544  -0.3231  -4.5876
1152   Topic4   12.395095        research   86.004653  -0.1524  -4.8607
1253   Topic4   10.898114         speaker   74.616334  -0.1390  -4.9894
446    Topic4   10.777180           event   74.528479  -0.1490  -5.0006
455    Topic4    5.701649        

#### Topic groups (4)
1. research
2. networking
3. day
4. variety

#### Topic groups (6) - 2 intersects with 1 & 4
1. think
2. practical
3. learn
4. design
5. variety
6. intimate

#### Topic groups (8) - overlap between 1 & 2 and 3 & 4
1. strive
2. world
3. concept
4. uxpa
5. immediately
6. sxsw
7. relevant
8. start


In [67]:
lda_W15 = lda15.transform(best_event_matrix)

In [68]:
top_doc_column15 = datadf.best_event.dropna()

In [69]:
word_count_matrix15, count_vect15 = nlp.create_wordcount_matrix(datadf.best_event.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [70]:
LDA15a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA15a.fit(word_count_matrix15)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [71]:
lda_H = LDA15a.transform(word_count_matrix15)

In [72]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column15, 3)

Top 3 Documents for Topic 0: 

Document 1
two polar opposite experience c2mtl highly produced inspiring talk specific networking and meeting discussion and opportunity in an adjacent area to my own not my core sme so not the same old people i used what i learned from that experience to design an incredible offsite for my organization a few month after i returned a 2day immersive workshopclass on analyzing data that i took from iit many year ago it wa full of tool some of which were new to me and handson feedback not only did i walk away with tool and skill that i didn't have before i used that experience to design immersive learning course in the future it wa one of the earliest learnbydoing class that i'd taken

Document 2
the best experience i had not ux focussed wa the impact factory training workshop that wa over 2 day called 'train the trainer' fantastic and engaging trainer and really interactive session with no slide deck at all i liked the ucd london conference in 2010 or 2011 

## 16. What if any events have you attended on the subject of research in the past few years?

In [73]:
datadf.events_attend_recent = datadf.events_attend_recent.astype('str')

In [74]:
datadf.events_attend_recent = datadf.events_attend_recent.dropna().apply(nlp.basic_clean)
datadf.events_attend_recent = datadf.events_attend_recent.dropna().apply(nlp.lemmatize)

In [75]:
events_attend_recent_matrix, events_attend_recent_vector = nlp.create_wordcount_matrix(datadf.events_attend_recent.dropna(), ngram=(1,3), max_df=.3)
events_attend_recent_matrix, events_attend_recent_vector

(<726x641 sparse matrix of type '<class 'numpy.int64'>'
 	with 3538 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [371]:
lda16 = LatentDirichletAllocation(n_components= 8, random_state = 42)

In [372]:
lda16.fit(events_attend_recent_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=8, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [373]:
pyLDAvis.sklearn.prepare(lda16, events_attend_recent_matrix, events_attend_recent_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.034696  0.054071       1        1  16.988362
5     -0.059680  0.041988       2        1  14.016953
2      0.212316 -0.170790       3        1  13.487871
4      0.093900  0.109352       4        1  12.962522
6     -0.045072  0.016259       5        1  12.074905
3     -0.047604 -0.009310       6        1  11.199995
0     -0.189840 -0.141092       7        1   9.847904
1      0.001285  0.099522       8        1   9.421489, topic_info=    Category        Freq                  Term       Total  loglift  logprob
351  Default  131.000000                   nan  131.000000  30.0000  30.0000
167  Default   41.000000                  epic   41.000000  29.0000  29.0000
561  Default   25.000000                  user   25.000000  28.0000  28.0000
562  Default   20.000000         user research   20.000000  27.0000  27.0000
567  Default  131.000000                    ux  131.000000  26.0000  26.0000
437  Default   16.000000       research london   16.000000  25.0000  25.0000
102  Default  132.000000            conference  132.000000  24.0000  24.0000
599  Default   46.000000                  uxpa   46.000000  23.0000  23.0000
563  Default   15.000000  user research london   15.000000  22.0000  22.0000
294  Default   32.000000                london   32.000000  21.0000  21.0000
428  Default  162.000000              research  162.000000  20.0000  20.0000
625  Default   52.000000              workshop   52.000000  19.0000  19.0000
37   Default   23.000000              attended   23.000000  18.0000  18.0000
517  Default   22.000000                strive   22.000000  17.0000  17.0000
277  Default   66.000000                 local   66.000000  16.0000  16.0000
195  Default   12.000000               focused   12.000000  15.0000  15.0000
408  Default   16.000000                  qrca   16.000000  14.0000  14.0000
136  Default   23.000000                   day   23.000000  13.0000  13.0000
65   Default   12.000000                boston   12.000000  12.0000  12.0000
320  Default   70.000000               meetups   70.000000  11.0000  11.0000
282  Default   20.000000         local meetups   20.000000  10.0000  10.0000
84   Default   18.000000                   chi   18.000000   9.0000   9.0000
133  Default   10.000000                course   10.000000   8.0000   8.0000
476  Default   19.000000        service design   19.000000   7.0000   7.0000
176  Default   45.000000                 event   45.000000   6.0000   6.0000
638  Default   20.000000                  year   20.000000   5.0000   5.0000
370  Default    7.000000                   nux    7.000000   4.0000   4.0000
9    Default   14.000000                  2019   14.000000   3.0000   3.0000
475  Default   20.000000               service   20.000000   2.0000   2.0000
601  Default    8.000000           uxpa boston    8.000000   1.0000   1.0000
..       ...         ...                   ...         ...      ...      ...
548   Topic8    1.830420          tomer sharon    2.578144   2.0197  -5.2625
484   Topic8    1.830420                sharon    2.578144   2.0197  -5.2625
453   Topic8    1.830420        research tomer    2.578144   2.0197  -5.2625
553   Topic8    1.830420               tuesday    2.578144   2.0197  -5.2625
456   Topic8    1.830420      research tuesday    2.578144   2.0197  -5.2625
514   Topic8    1.830420          storytelling    2.578144   2.0197  -5.2625
565   Topic8    1.830382               usually    2.578140   2.0196  -5.2625
186   Topic8    1.830372        field research    2.578144   2.0196  -5.2625
621   Topic8    1.830372               webexpo    2.578145   2.0196  -5.2625
342   Topic8    1.830364          mixed method    2.578144   2.0196  -5.2626
274   Topic8    3.553315                  like    5.075835   2.0056  -4.5992
572   Topic8    3.553237               ux camp    5.161918   1.9887  -4.5992
110   Topic8    2.691880    c

# Topic groups (4)

1. day
2. london
3. epic
4. uxpa

#### Topic groups (6)
1. london
2. summit
3. session
4. design research
5. epic
6. uxr

#### Topic groups (8)
Heavy overlap among 6 of the 8 topics
1. local meetups
2. qrca
3. design research
4. uxpa
5. attended
6. epic
7. user research
8. focused


In [79]:
lda_W16 = lda16.transform(events_attend_recent_matrix)

In [80]:
top_doc_column16 = datadf.events_attend_recent.dropna()

In [81]:
word_count_matrix16, count_vect16 = nlp.create_wordcount_matrix(datadf.events_attend_recent.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [82]:
LDA16a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA16a.fit(word_count_matrix16)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [83]:
lda_H = LDA16a.transform(word_count_matrix16)

In [84]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column16, 3)

Top 3 Documents for Topic 0: 

Document 1
uxr collective's strive conference in toronto

Document 2
nux manchester steve portigal's research networking dinner meetup at answerlab how to minimize research bias in your project

Document 3
strive local uxpa chapter meeting boston uxpa conference

Top 3 Documents for Topic 1: 

Document 1
researchops workshop user research sydney user research london ux insight utrecht

Document 2
research ops remote meetups ux monday in prague research tuesday in prague some ux conference meetups and barcamps in berlin bratislava brno prague and vienna

Document 3
not really research only but when i go to ux or web conference i pick research speaches nng exconf bratislava doer budapest webexpo prague ux salon tel aviv ux camp europe in berlin best thing ever mostly and i have a ticket for ux brighton

Top 3 Documents for Topic 2: 

Document 1
a 2 day ux design and research course numerous monthly nux event an nux conference in manchester

Document 2
actua

## 20. Did we miss any other types of conference sessions that you'd like to mention?

In [85]:
datadf.other_conference_types = datadf.other_conference_types.dropna().apply(nlp.basic_clean)
datadf.other_conference_types = datadf.other_conference_types.dropna().apply(nlp.lemmatize)

In [86]:
other_conference_types_matrix, other_conference_types_vector = nlp.create_wordcount_matrix(datadf.other_conference_types.dropna(), ngram=(1,3), max_df=.3)
other_conference_types_matrix, other_conference_types_vector

(<258x295 sparse matrix of type '<class 'numpy.int64'>'
 	with 1136 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [384]:
lda20 = LatentDirichletAllocation(n_components= 8, random_state = 42)

In [385]:
lda20.fit(other_conference_types_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=8, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [386]:
pyLDAvis.sklearn.prepare(lda20, other_conference_types_matrix, other_conference_types_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.009715 -0.006929       1        1  25.951566
3     -0.067333 -0.074572       2        1  12.092869
5     -0.007659  0.002315       3        1  11.706542
6      0.042670 -0.117714       4        1  11.319701
4      0.033914 -0.099119       5        1  10.654923
0      0.063353  0.082214       6        1  10.485077
2     -0.205937  0.088453       7        1   9.660771
1      0.131277  0.125351       8        1   8.128551, topic_info=    Category       Freq                 Term      Total  loglift  logprob
159  Default  14.000000                 nope  14.000000  30.0000  30.0000
216  Default  34.000000              session  34.000000  29.0000  29.0000
256  Default  18.000000                think  18.000000  28.0000  28.0000
289  Default  20.000000             workshop  20.000000  27.0000  27.0000
166  Default   7.000000                panel   7.000000  26.0000  26.0000
179  Default   8.000000               poster   8.000000  25.0000  25.0000
22   Default   7.000000                 case   7.000000  24.0000  24.0000
264  Default   8.000000                track   8.000000  23.0000  23.0000
23   Default   5.000000           case study   5.000000  22.0000  22.0000
172  Default  25.000000               people  25.000000  21.0000  21.0000
241  Default   6.000000                study   6.000000  20.0000  20.0000
63   Default   7.000000                  don   7.000000  19.0000  19.0000
180  Default   5.000000       poster session   5.000000  18.0000  18.0000
204  Default  18.000000             research  18.000000  17.0000  17.0000
286  Default   9.000000                 work   9.000000  16.0000  16.0000
33   Default  30.000000           conference  30.000000  15.0000  15.0000
153  Default   4.000000                   na   4.000000  14.0000  14.0000
255  Default   5.000000                thing   5.000000  13.0000  13.0000
246  Default  14.000000                 talk  14.000000  12.0000  12.0000
284  Default  13.000000                  way  13.000000  11.0000  11.0000
58   Default   6.000000           discussion   6.000000  10.0000  10.0000
12   Default   7.000000             attendee   7.000000   9.0000   9.0000
167  Default   3.000000     panel discussion   3.000000   8.0000   8.0000
106  Default   6.000000                 idea   6.000000   7.0000   7.0000
276  Default   4.000000              variety   4.000000   6.0000   6.0000
64   Default   3.000000            don think   3.000000   5.0000   5.0000
259  Default   8.000000                 time   8.000000   4.0000   4.0000
28   Default   4.000000                 come   4.000000   3.0000   3.0000
70   Default  14.000000                event  14.000000   2.0000   2.0000
261  Default   4.000000                 tool   4.000000   1.0000   1.0000
..       ...        ...                  ...        ...      ...      ...
182   Topic8   1.521632            practical   3.804194   1.5935  -4.1941
232   Topic8   0.805599                 sort   2.328238   1.4485  -4.8301
287   Topic8   0.805569        work progress   2.328246   1.4485  -4.8301
181   Topic8   0.805564           powerpoint   2.329686   1.4478  -4.8301
37    Topic8   0.805558  conference workshop   2.329687   1.4478  -4.8301
65    Topic8   0.805551                dunno   2.329688   1.4478  -4.8301
290   Topic8   0.805580  workshop conference   2.396910   1.4194  -4.8301
67    Topic8   0.805573             engaging   2.396912   1.4194  -4.8301
273   Topic8   0.805564                using   2.396914   1.4194  -4.8301
95    Topic8   1.521719              handson   4.805820   1.3598  -4.1940
286   Topic8   2.953838                 work   9.592166   1.3319  -3.5308
268   Topic8   1.521604         unconference   5.358404   1.2509  -4.1941
16    Topic8   0.805593               better   3.125440   1.1540  -4.8301
84    Topic8   0.805619                  fun   3.155376   1.1445  -4.8300
190   Topic8 

#### Topic groups (4)
1. case study
2. talk
3. poster session
4. nope

#### Topic groups (6)
1. reatreat
2. working
3. quality
4. case study
5. panel discussion
6. tutorial

#### Topic groups (8) - 3 is completely inside 1; 4 and 5 share about 75% of the same area
1. working
2. case study
3. nice
4. outside
5. variety
6. multitrack
7. panel
8. method

In [90]:
lda_W20 = lda20.transform(other_conference_types_matrix)

In [91]:
top_doc_column20 = datadf.other_conference_types.dropna()

In [92]:
word_count_matrix20, count_vect20 = nlp.create_wordcount_matrix(datadf.other_conference_types.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [93]:
LDA20a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA20a.fit(word_count_matrix20)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [94]:
lda_H = LDA20a.transform(word_count_matrix20)

In [95]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column20, 3)

Top 3 Documents for Topic 0: 

Document 1
i am there to learn efficiently not socialize so the thing you didn't mention is the availability of course note at least the speakers' slide that i can study before and after the talk i have also been at conference that have a knowledgeable discussant associated with each 1hour talk the discussant ha studied the work before and asks pointed question about pro and con relationship to other technique point of failure etc when the speaker know this is coming you are le likely to get just a canned talk and you really get to the meat of the issue

Document 2
to make a conference really be incredible making something that they take away is critical cocreation is a good way to crystalize learning and having a physical artifact to take away and help you remember what you learned is great now that i wrote that i think it might be pretty cool to do a handson session on sketchnoting at the beginning of the conference so people who haven't ever done it be

## 21. Subjects you most want to see covered at a research conference

In [96]:
datadf.ideal_topics = datadf.ideal_topics.dropna().apply(nlp.basic_clean)
datadf.ideal_topics = datadf.ideal_topics.dropna().apply(nlp.lemmatize)

In [97]:
ideal_topics_matrix, ideal_topics_vector = nlp.create_wordcount_matrix(datadf.ideal_topics.dropna(), ngram=(1,3), max_df=.3)
ideal_topics_matrix, ideal_topics_vector

(<560x1122 sparse matrix of type '<class 'numpy.int64'>'
 	with 5854 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [387]:
lda21 = LatentDirichletAllocation(n_components= 4, random_state = 42)

In [388]:
lda21.fit(ideal_topics_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=4, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [389]:
pyLDAvis.sklearn.prepare(lda21, ideal_topics_matrix, ideal_topics_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.149435  0.004902       1        1  28.179038
1     -0.063588 -0.062169       2        1  25.641587
2     -0.030393 -0.060260       3        1  24.997328
0     -0.055453  0.117526       4        1  21.182047, topic_info=     Category        Freq                 Term       Total  loglift  logprob
989   Default   58.000000                study   58.000000  30.0000  30.0000
132   Default   51.000000                 case   51.000000  29.0000  29.0000
134   Default   43.000000           case study   43.000000  28.0000  28.0000
582   Default  126.000000               method  126.000000  27.0000  27.0000
663   Default   28.000000         organization   28.000000  26.0000  26.0000
221   Default   46.000000                 data   46.000000  25.0000  25.0000
351   Default   19.000000              finding   19.000000  24.0000  24.0000
654   Default   20.000000                  ops   20.000000  23.0000  23.0000
632   Default   14.000000           new method   14.000000  22.0000  22.0000
627   Default   61.000000                  new   61.000000  21.0000  21.0000
37    Default   14.000000                   ai   14.000000  20.0000  20.0000
826   Default   11.000000     research finding   11.000000  19.0000  19.0000
853   Default   15.000000         research ops   15.000000  18.0000  18.0000
720   Default   12.000000         presentation   12.000000  17.0000  17.0000
737   Default   18.000000              project   18.000000  16.0000  16.0000
755   Default   16.000000                quant   16.000000  15.0000  15.0000
968   Default   22.000000          stakeholder   22.000000  14.0000  14.0000
744   Default   15.000000                 qual   15.000000  13.0000  13.0000
315   Default   26.000000                ethic   26.000000  12.0000  12.0000
58    Default   14.000000          application   14.000000  11.0000  11.0000
1114  Default   12.000000              working   12.000000  10.0000  10.0000
707   Default   19.000000            practical   19.000000   9.0000   9.0000
611   Default   10.000000         mixed method   10.000000   8.0000   8.0000
610   Default   10.000000                mixed   10.000000   7.0000   7.0000
985   Default   21.000000             strategy   21.000000   6.0000   6.0000
185   Default    9.000000              content    9.000000   5.0000   5.0000
126   Default   10.000000                buyin   10.000000   4.0000   4.0000
349   Default   23.000000                field   23.000000   3.0000   3.0000
600   Default   25.000000          methodology   25.000000   2.0000   2.0000
622   Default   17.000000                 need   17.000000   1.0000   1.0000
...       ...         ...                  ...         ...      ...      ...
582    Topic4   53.245653               method  126.501881   0.6867  -3.2073
58     Topic4    8.417528          application   14.284892   1.0231  -5.0519
1029   Topic4    7.392382                think   12.538025   1.0237  -5.1818
121    Topic4   10.022700             business   20.122136   0.8550  -4.8774
727    Topic4    8.340565              problem   15.947055   0.9039  -5.0611
627    Topic4   21.828669                  new   61.422846   0.5175  -4.0990
528    Topic4    4.350216                 life    6.691423   1.1214  -5.7120
529    Topic4   11.765331                 like   28.603872   0.6636  -4.7171
441    Topic4    6.663046             industry   12.572162   0.9171  -5.2856
916    Topic4    5.995218              scaling   10.871570   0.9568  -5.3913
471    Topic4    4.281316          interesting    6.694548   1.1050  -5.7280
510    Topic4    3.517820  leadership research    5.003671   1.1997  -5.9244
262    Topic4    3.516332            difficult    5.001835   1.1996  -5.9248
634    Topic4    3.511146         new research    5.003830   1.1978  -5.9263
668    Topic4    3.509937                 orgs    5.003876   1.1974  -5.9266
509    Topic4 

#### Topic groups (4) - considerable overlap between 2 & 3
1. case study
2. finding
3. ops
4. new method

#### Topic groups (6)
1. case study
2. ops
3. stakeholder
4. new method
5. user research
6. quantitative

#### Topic groups (8)
1. case study
2. ops
3. user research
4. working
5. quant
6. mixed method
7. ethic
8. qualitative data


In [101]:
lda_W21 = lda21.transform(ideal_topics_matrix)

In [102]:
top_doc_column21 = datadf.ideal_topics.dropna()

In [103]:
word_count_matrix21, count_vect21 = nlp.create_wordcount_matrix(datadf.ideal_topics.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [104]:
LDA21a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA21a.fit(word_count_matrix21)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [105]:
lda_H = LDA21a.transform(word_count_matrix21)

In [106]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column21, 3)

Top 3 Documents for Topic 0: 

Document 1
researchops integrating research with design product and engineering growing and scaling a research practice leading a research team enterprise org structure for research team scaling the impact of research across an org research with emerging tech aimachine learning mrvrar multimodal experience etc

Document 2
facilitation skill conversation design master class teaching research basic to others skill for new research manager former ic moving into research management mixed method research design lightning improv shareout how to setup ux dashboard cxos qualinformed touchpoint specific outcome that matter most to people and ongoing quant measurement of those key experiencesmoments ethicalmoral framework for research that advance ai how do we consider downstream impact and just because we can doesnt mean we should

Document 3
i'd like to see if organization are still focusing on usability in my recent role there ha no longer been an emphasis on th

## 22. If attending a conference about research, who might you be excited to see there?

In [107]:
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.basic_clean)
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.lemmatize)

In [108]:
ideal_att_matrix, ideal_att_vector = nlp.create_wordcount_matrix(datadf.ideal_attendees.dropna(), ngram=(1,3), max_df=.3)
ideal_att_matrix, ideal_att_vector

(<471x923 sparse matrix of type '<class 'numpy.int64'>'
 	with 4277 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [394]:
lda22 = LatentDirichletAllocation(n_components= 4, random_state = 42)

In [395]:
lda22.fit(ideal_att_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=4, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [396]:
pyLDAvis.sklearn.prepare(lda22, ideal_att_matrix, ideal_att_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.099256  0.012285       1        1  26.978010
3      0.068893 -0.087438       2        1  25.507302
2     -0.020914  0.125629       3        1  25.495105
1     -0.147236 -0.050476       4        1  22.019582, topic_info=    Category       Freq              Term      Total  loglift  logprob
353  Default  40.000000          industry  40.000000  30.0000  30.0000
920  Default  16.000000             young  16.000000  29.0000  29.0000
514  Default  18.000000               new  18.000000  28.0000  28.0000
226  Default  14.000000             erika  14.000000  27.0000  27.0000
349  Default  12.000000        indi young  12.000000  26.0000  26.0000
348  Default  12.000000              indi  12.000000  25.0000  25.0000
227  Default  13.000000        erika hall  13.000000  24.0000  24.0000
264  Default  31.000000             field  31.000000  23.0000  23.0000
784  Default  22.000000             steve  22.000000  22.0000  22.0000
244  Default  22.000000        experience  22.000000  21.0000  21.0000
308  Default  15.000000              hall  15.000000  20.0000  20.0000
532  Default   8.000000            norman   8.000000  19.0000  19.0000
830  Default   8.000000             tomer   8.000000  18.0000  18.0000
406  Default  15.000000            ladner  15.000000  17.0000  17.0000
705  Default  15.000000               sam  15.000000  16.0000  16.0000
706  Default  15.000000        sam ladner  15.000000  15.0000  15.0000
687  Default   9.000000     research team   9.000000  14.0000  14.0000
601  Default  18.000000          portigal  18.000000  13.0000  13.0000
464  Default  11.000000              love  11.000000  12.0000  12.0000
811  Default  20.000000              tech  20.000000  11.0000  11.0000
734  Default   7.000000            sharon   7.000000  10.0000  10.0000
786  Default  17.000000    steve portigal  17.000000   9.0000   9.0000
422  Default   8.000000      leader field   8.000000   8.0000   8.0000
419  Default  54.000000            leader  54.000000   7.0000   7.0000
445  Default  43.000000              like  43.000000   6.0000   6.0000
153  Default  11.000000              data  11.000000   5.0000   5.0000
831  Default   6.000000      tomer sharon   6.000000   4.0000   4.0000
520  Default   6.000000           nielsen   6.000000   3.0000   3.0000
544  Default  32.000000      organization  32.000000   2.0000   2.0000
650  Default   9.000000              real   9.000000   1.0000   1.0000
..       ...        ...               ...        ...      ...      ...
293   Topic4   2.637633  google microsoft   3.270208   1.2983  -5.9249
812   Topic4   5.911240      tech company   7.362999   1.2936  -5.1180
425   Topic4   5.088393           leading   6.543947   1.2617  -5.2678
509   Topic4   1.830903         nate bolt   2.456270   1.2194  -6.2900
718   Topic4   1.830903             scott   2.456270   1.2194  -6.2900
865   Topic4   5.861215     user research   8.188759   1.1788  -5.1265
784   Topic4  14.034420             steve  22.179972   1.0556  -4.2533
601   Topic4  10.760934          portigal  18.085203   0.9941  -4.5189
35    Topic4   4.163990    anthropologist   5.738417   1.1925  -5.4683
786   Topic4  10.012089    steve portigal  17.270355   0.9680  -4.5910
406   Topic4   8.592003            ladner  15.663440   0.9127  -4.7440
705   Topic4   8.592003               sam  15.663440   0.9127  -4.7440
706   Topic4   8.592003        sam ladner  15.663440   0.9127  -4.7440
773   Topic4   7.349209             spool  13.995644   0.8691  -4.9002
339   Topic4   4.294067              ideo   6.565608   1.0886  -5.4376
202   Topic4   8.297746               don  17.304115   0.7783  -4.7788
811   Topic4   8.716443              tech  20.675885   0.6495  -4.7296
112   Topic4  13.352545           company  49.728987   0.1984  -4.3031
547   Topic4   6.047930              orgs  11.536891   0.8674  -5.0951
783   Topic4

#### Topic groups (4)
1. industry
2. field
3. erika hall
4. Indi young

#### Topic groups (6) - some overlap between 2, 3, & 4; 5 is mostly overlapped with 2 & 4
1. steve portigal
2. expert
3. diverse
4. new
5. startup
6. doing research

#### Topic groups (8)
1. google
2. jared spool
3. different
4. new
5. consultant
6. erika hall
7. practioner
8. steve portigal


In [112]:
lda_W22 = lda22.transform(ideal_att_matrix)

In [113]:
top_doc_column22 = datadf.ideal_attendees.dropna()

In [114]:
word_count_matrix22, count_vect22 = nlp.create_wordcount_matrix(datadf.ideal_attendees.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [115]:
LDA22a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA22a.fit(word_count_matrix22)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [116]:
lda_H = LDA22a.transform(word_count_matrix22)

In [117]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column22, 3)

Top 3 Documents for Topic 0: 

Document 1
people with experience in building company by applying research knowledge i think there are people out there speaking from a consulting view they donat take a many risk a the one iad like to learn from company are usually led by people with a techdesign background if research is creating a real value in company and if we believe research can lead the strategic vision of a company iall be excited to see people who have managed to achieve that

Document 2
i would like to hear from and network with other organizational leader i want to meet people i can learn from no pr or salestype presentation no product pushing although i'd like to learn about tool that people are using i am much more interested in strategy and how to manage operation for large team also speaking a someone on the consulting side it would be nice to hear from leader in that space a well a leader within organization some of the advice from design executive doesn't really apply to

In [383]:
datadf.recommendations = datadf.recommendations.dropna().apply(nlp.basic_clean)
datadf.recommendations = datadf.recommendations.dropna().apply(nlp.lemmatize)

In [382]:
recommendations_matrix, recommendations_vector = nlp.create_wordcount_matrix(datadf.recommendations.dropna(), ngram=(1,3), max_df=.3)
recommendations_matrix, recommendations_vector

(<494x1409 sparse matrix of type '<class 'numpy.int64'>'
 	with 7225 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [407]:
lda23 = LatentDirichletAllocation(n_components= 8, random_state = 42)

In [408]:
lda23.fit(recommendations_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=8, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [409]:
pyLDAvis.sklearn.prepare(lda23, recommendations_matrix, recommendations_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.075875 -0.005570       1        1  18.590069
5      0.061837  0.052963       2        1  17.491367
6      0.058806  0.004421       3        1  15.921481
3      0.003337 -0.031329       4        1  14.333301
2     -0.007261 -0.016992       5        1  13.461363
1      0.020556  0.025219       6        1   8.967949
4     -0.077382 -0.129276       7        1   6.142941
0     -0.135768  0.100564       8        1   5.091529, topic_info=     Category        Freq             Term       Total  loglift  logprob
735   Default  118.000000             make  118.000000  30.0000  30.0000
528   Default   22.000000             help   22.000000  29.0000  29.0000
895   Default  130.000000           people  130.000000  28.0000  28.0000
217   Default  115.000000       conference  115.000000  27.0000  27.0000
77    Default   35.000000           attend   35.000000  26.0000  26.0000
497   Default   29.000000            great   29.000000  25.0000  25.0000
348   Default   73.000000              don   73.000000  24.0000  24.0000
736   Default    9.000000  make accessible    9.000000  23.0000  23.0000
1327  Default   28.000000               ux   28.000000  22.0000  22.0000
12    Default   17.000000       accessible   17.000000  21.0000  21.0000
35    Default   10.000000       affordable   10.000000  20.0000  20.0000
769   Default   14.000000            maybe   14.000000  19.0000  19.0000
947   Default   17.000000        practical   17.000000  18.0000  18.0000
400   Default   23.000000           events   23.000000  17.0000  17.0000
489   Default   40.000000             good   40.000000  16.0000  16.0000
1397  Default   17.000000            world   17.000000  15.0000  15.0000
193   Default   11.000000             come   11.000000  14.0000  14.0000
396   Default   10.000000           europe   10.000000  13.0000  13.0000
1116  Default   22.000000         sessions   22.000000  12.0000  12.0000
1193  Default   10.000000            stuff   10.000000  11.0000  11.0000
834   Default    5.000000            north    5.000000  10.0000  10.0000
1329  Default   10.000000      ux research   10.000000   9.0000   9.0000
844   Default    6.000000         offering    6.000000   8.0000   8.0000
1224  Default   27.000000             talk   27.000000   7.0000   7.0000
230   Default   65.000000      conferences   65.000000   6.0000   6.0000
1146  Default   12.000000          smaller   12.000000   5.0000   5.0000
1362  Default   28.000000             want   28.000000   4.0000   4.0000
1067  Default   49.000000      researchers   49.000000   3.0000   3.0000
458   Default   15.000000             food   15.000000   2.0000   2.0000
710   Default   12.000000             look   12.000000   1.0000   1.0000
...       ...         ...              ...         ...      ...      ...
1155   Topic8    0.808128           spaces    2.353252   1.9088  -6.2079
15     Topic8    0.808122              act    2.353253   1.9088  -6.2079
181    Topic8    0.808121  city conference    2.353253   1.9088  -6.2079
193    Topic8    3.681866             come   11.203241   1.8648  -4.6915
528    Topic8    6.923221             help   22.873666   1.7825  -4.0600
1102   Topic8    1.526172             sell    4.811921   1.8293  -5.5721
1185   Topic8    1.526606           strive    4.856192   1.8204  -5.5718
1248   Topic8    1.526676           theory    4.866986   1.8182  -5.5718
769    Topic8    3.927078            maybe   14.851469   1.6474  -4.6270
497    Topic8    6.902519            great   29.733630   1.5172  -4.0630
64     Topic8    2.245028             area    8.116479   1.6924  -5.1862
396    Topic8    2.629422           europe   10.206414   1.6213  -5.0281
710    Topic8    2.963444             look   12.981362   1.5004  -4.9085
895    Topic8   12.152369           people  130.841813   0.6011  -3.4973
217    Topic8    7.394101       conference  115.865740   0.

#### Topic groups (4)
1. pratical
2. affordable
3. expensive
4. levels

#### Topic groups (6) - some overlap between 2, 3, & 4; 5 is mostly overlapped with 2 & 4
1. ux
2. interesting
3. content
4. food
5. learning
6. affordalbe

#### Topic groups (8) - significant overlap amoung 6 of the 8 groups
1. events
2. ux
3. world
4. new
5. food
6. diverse
7. make accessible
8. north america

In [410]:
lda_W23 = lda23.transform(recommendations_matrix)

In [411]:
top_doc_column23 = datadf.recommendations.dropna()

In [412]:
word_count_matrix23, count_vect23 = nlp.create_wordcount_matrix(datadf.recommendations.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [413]:
LDA23a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA23a.fit(word_count_matrix23)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [414]:
lda23.fit(recommendations_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=8, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [415]:
lda_H = LDA23a.transform(word_count_matrix23)

In [416]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column23, 3)

Top 3 Documents for Topic 0: 

Document 1
the deeper you go into the subject matter the better a lot of research literature is either design adjacent or academic focusing on a research practitioner daytoday someone that is in a software company or an agency or a startup would be super helpful anything that help me grow my skill and competency to ultimately be a better researcher

Document 2
put together a draft conference agenda go out to the community and research what option people desire the most use the insight to inform a single track conference which you can take around the world to multiple location so it accessible on a global scale keep it small and intimate make it feel like a community event repeat each year

Document 3
ha ha ha i wonder how this would be similar to or different from epic i like the idea of a small intimate conference that focus on interactive session and workshop rather than paper there is a place for that too but i feel there are enough of those i'd love a